In [ ]:
import joblib
storage_account_name = "<your storage_account_name>"
sas_token = "<your SAS token>"

spark.conf.set(f"fs.azure.account.auth.type.{storage_account_name}.dfs.core.windows.net", "SAS")
spark.conf.set(f"fs.azure.sas.token.provider.type.{storage_account_name}.dfs.core.windows.net", 
               "org.apache.hadoop.fs.azurebfs.sas.FixedSASTokenProvider")
spark.conf.set(f"fs.azure.sas.fixed.token.{storage_account_name}.dfs.core.windows.net", sas_token)

# 1. Copy from ADLS back to local driver:
dbutils.fs.cp(
    "abfss://{your first container name}@{storage_account_name}.dfs.core.windows.net/cpg_forecast_rf.pkl",
    "file:/tmp/cpg_forecast_rf.pkl"
)

# 2. Load it with joblib
model = joblib.load("/tmp/cpg_forecast_rf.pkl")

# 3. Confirm it's loaded
print("Model loaded successfully!")


In [ ]:


future_path = "abfss://{your first container name}@{storage_account_name}.dfs.core.windows.net/future_data.csv"
df_future = spark.read.option("header","true").csv(future_path)
df_future.show(5)


In [ ]:
from pyspark.sql.functions import dayofweek, weekofyear, month, year, to_date, col, when

df_future = df_future.withColumn("date", to_date(col("date"), "yyyy-MM-dd")) \
                     .withColumn("day_of_week", dayofweek(col("date"))) \
                     .withColumn("week_of_year", weekofyear(col("date"))) \
                     .withColumn("month", month(col("date"))) \
                     .withColumn("year", year(col("date")))

# If you have a holiday or promotion logic, replicate it here
df_future = df_future.withColumn("is_holiday", when(col("month") == 1, 1).otherwise(0)) 


In [ ]:
df_future_pd = df_future.toPandas()

feature_cols = [
   "day_of_week", "week_of_year", "month", "year",
   "is_holiday", "is_promo_active", 
   "price"
   # If you are skipping rolling_7d_qty for future predictions, remove it
]

X_future = df_future_pd[feature_cols]


In [ ]:
y_pred_future = model.predict(X_future)

df_future_pd["predicted_sales_qty"] = y_pred_future

In [ ]:
df_preds_spark = spark.createDataFrame(df_future_pd)

In [ ]:
output_path = f"abfss://{your new container name for storing prediction}@{storage_account_name}.dfs.core.windows.net/future_predictions.parquet"
df_preds_spark.write.mode("overwrite").parquet(output_path)